In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns

def prepare_data(df):
    # Drop unnecessary columns
    df = df.drop(["transaction_id", "date"], axis=1)

    # Encode categorical features
    categorical_cols = ["card_type", "merchant", "gender"]
    for col in categorical_cols:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col])

    # Scale numerical features
    numerical_cols = ["amount", "age", "income"]
    scaler = StandardScaler()
    df[numerical_cols] = scaler.fit_transform(df[numerical_cols])
    
    return df

def fraud_detection_model(df):
    print("\n--- Fraud Detection Model ---")
    X = df.drop("is_fraud", axis=1)
    y = df["is_fraud"]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))
    print("\nConfusion Matrix:")
    print(confusion_matrix(y_test, y_pred))

    # Feature Importance
    feature_importances = pd.Series(model.feature_importances_, index=X.columns).sort_values(ascending=False)
    print("\nFeature Importances:")
    print(feature_importances)

def customer_segmentation_model(df):
    print("\n--- Customer Segmentation Model ---")
    # For segmentation, we will use customer-level data, so aggregate transactions
    customer_df = df.groupby("customer_id").agg({
        "amount": "sum",
        "age": "first",
        "income": "first",
        "is_fraud": "sum" # Number of fraudulent transactions per customer
    }).reset_index()

    # Select features for clustering
    X_segmentation = customer_df[["amount", "age", "income"]]

    # Determine optimal number of clusters using Elbow Method
    wcss = []
    for i in range(1, 11):
        kmeans = KMeans(n_clusters=i, init='k-means++', random_state=42, n_init=10)
        kmeans.fit(X_segmentation)
        wcss.append(kmeans.inertia_)

    plt.figure(figsize=(10, 6))
    plt.plot(range(1, 11), wcss, marker='o')
    plt.title("Elbow Method for Optimal K")
    plt.xlabel("Number of Clusters (K)")
    plt.ylabel("WCSS")
    plt.savefig("C:/Users/ajroy/OneDrive/Desktop/CREDIT/docs/elbow_method.png")
    plt.close()

    # Apply KMeans with chosen K (e.g., K=3 based on typical elbow)
    optimal_k = 3 # This would be determined by visual inspection of the elbow plot
    kmeans = KMeans(n_clusters=optimal_k, init='k-means++', random_state=42, n_init=10)
    customer_df["cluster"] = kmeans.fit_predict(X_segmentation)

    print("\nCustomer Segments (first 5 rows with cluster):")
    print(customer_df.head())

    # Visualize clusters
    plt.figure(figsize=(10, 8))
    sns.scatterplot(x="amount", y="income", hue="cluster", data=customer_df, palette="viridis")
    plt.title("Customer Segments")
    plt.savefig("C:/Users/ajroy/OneDrive/Desktop/CREDIT/docs/customer_segments.png")
    plt.close()

    # Interpret model results (print to console for now)
    print("\n--- Model Interpretation ---")
    print("Fraud Detection Model: Random Forest performed well in identifying fraudulent transactions.")
    print("Customer Segmentation Model: Customers are grouped into distinct segments based on their spending and demographic data.")

file_path = "C:/Users/ajroy/OneDrive/Desktop/CREDIT/DATA/credit_card_transactions.csv"
df = pd.read_csv(file_path)

processed_df = prepare_data(df.copy())
fraud_detection_model(processed_df.copy())
customer_segmentation_model(processed_df.copy())
print("Machine learning models trained and evaluated. Visualizations saved to CREDIT/docs/")